<font size="18">Chain Gang on a Fence</font>

In [ ]:
# set this notebook to use a large part of the browser window width
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:80% !important; }</style>"))

# Introduction

We consider the setup where N people are holding hands in a chain, so in series and the first person
puts one hand on an electric fence.

Given normal human resistances, we intend here to calculate the currents through arms and bodies of each person.

This is mainly to refute a false belief 
stated on [this Quora webpage](https://www.quora.com/If-a-line-of-people-hold-hands-and-the-person-at-the-end-touches-an-electric-fence-who-will-get-the-greatest-shock)
that somehow, the last person in the chain would receives the most current through his arms or body.
This belief makes N-1 people play some pranks on the Nth person, and they are wrongly convinced that the Nth person suffers the most. It must be group psychology, tricking the N-1 people into that.


# Model Setup

The network of resistances can be represented as in the below figure.

In [ ]:
from IPython.display import Image
Image(filename='ChainGangOfAFence.png', width=1000, height=3000)

Using Kirchhoff current and voltege laws we get the equations.

In [ ]:
Image(filename='KirchhoffCurrentLawUpperNodes.tex.png')

In [ ]:
Image(filename='KirchhoffCurrentLawLowerNodes.tex.png')

In [ ]:
Image(filename='KirchhoffVoltageLawForAllLoops.tex.png')

In [ ]:
Image(filename='VoltageToNextVoltageLawForAllLoops.tex.png')

This gives a system of 3*(N) + 1*(N+1), so 4N+1 linear equations 
for 3N current variables and N+1 voltage variables, so also 4N+1 variables, so that is nicely and easily solvable using some linear algebra.

Even though, there is only one solution here and we do not have an objective, to solve this,
we can still use the Integer Linear Programmnig (ILP) solver Gurobi. There will be only continuous variables
and the problem is a feasibility rather than optimisation problem.
We code this as follows.

In [ ]:
from gurobipy import *

def set_resistances(N, hand_to_hand_resistance_ohm = 10000, neck_to_feet_resistance_ohm = 10000):    
    verbose = 0

    person_thru_ground_to_prev_person_resistance_ohm = 10000
    r_a = [hand_to_hand_resistance_ohm] * N
    r_b = [neck_to_feet_resistance_ohm] * N
    r_g = [person_thru_ground_to_prev_person_resistance_ohm] * N
    
    if verbose >= 1:
        print(r_a)

    return r_a, r_b, r_g

def print_resistances(r_a, r_b, r_g):
    for const_name, const_struct in list(zip(['r_a', 'r_b', 'r_g'], [r_a, r_b, r_g])):
        print(const_name + ' (Ohm) = ')
        print(const_struct)

def calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000):
    verbose = 0
    
    N = len(r_a)
    assert len(r_b) == N
    assert len(r_g) == N
    
    m = Model()

    rN = range(0,N)         # [0, 1, .. N-1]
    rNplus1 = range(0,N+1)  # [0, 1, .. N]

    rCurrents = list(rN)
    rVoltages = list(rNplus1)  # we also define v0 as a variable even if this battery voltage is fixed and known ahead.
    
    if verbose >= 1:
        print(rCurrents)
        print(rVoltages)

    # create all the 4N continuous variables:
    i_a = m.addVars(rCurrents, vtype=GRB.CONTINUOUS, name='i_a')  # upper node currents (going to the right thru arms)
    i_b = m.addVars(rCurrents, vtype=GRB.CONTINUOUS, name='i_b')  # body currents (flowing down from nack to legs)
    i_g = m.addVars(rCurrents, vtype=GRB.CONTINUOUS, name='i_g')  # lower node to battery anode (in ground) currents
    v = m.addVars(rVoltages, vtype=GRB.CONTINUOUS, name='v')    # v0 and N voltages at person i neck for i in 1..N

    assert len(i_a) == N
    assert len(i_b) == N
    assert len(i_g) == N
    assert len(v) == N+1

    # create all the 4N linear constraints:
    for i in rCurrents:  # there are N of them: 0..N-1
        if i < N-1:
            m.addConstr(i_a[i  ] == i_a[i+1] + i_b[i  ], 'i_a_constr{:d}'.format(i))
            m.addConstr(i_g[i  ] == i_b[i  ] + i_g[i+1], 'i_b_constr{:d}'.format(i))
        else:
            m.addConstr(i_a[i  ] ==            i_b[i  ], 'i_a_constr{:d}'.format(i))
            m.addConstr(i_g[i  ] == i_b[i  ]           , 'i_b_constr{:d}'.format(i))

    for i in rVoltages: # there are N+1 of them: 0,..N
        if i < N:
            m.addConstr(v[i  ] - r_a[i] * i_a[i] - r_b[i] * i_b[i] - r_g[i] * i_g[i] == 0, 
                        'v_constr{:d}'.format(i))
            m.addConstr(v[i  ] - r_a[i] * i_a[i] == v[i+1], 
                        'v_rel{:d}'.format(i))

    # Set the battery voltage:
    m.addConstr(v[0] == battery_voltage, 'v_0_fix')  # 5000V is a typical average voltage for an
    # electric fence. See: https://hypertextbook.com/facts/2001/NicoleCastellano.shtml

    m.optimize()
    
    return m, i_a, i_b, i_g, v
        
def print_results(m, i_a, i_b, i_g, v):    
    for var_name, var_struct in list(zip(['i_a', 'i_b', 'i_g', 'v'], [i_a, i_b, i_g, v])):
        values = m.getAttr('x', var_struct)
        s = var_name
        if var_name != 'v':
            s += ' (mA) = '
            vals = ['{:d}'.format(int(values[v] * 1000)) for v in values]
        else:
            s += ' (kV) = '
            vals = ['{:.2f}'.format((values[v] / 1000)) for v in values]
        print(s)
        print(vals)

# Calculation Examples

## Example with 2 Persons

In [ ]:
N = 2  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, hand_to_hand_resistance_ohm = 10000, neck_to_feet_resistance_ohm = 10000)
m, i_a, i_b, i_g, v = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, v)

So all current series i_a and i_b (and even i_g) show lower and lower values, when going from the first to the last person.
This means that in all respects, in arms and body, the Nth person is the one suffering the least, contrary to the popular false belief.

## Example with 3 Persons

In [ ]:
N = 3  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, hand_to_hand_resistance_ohm = 10000, neck_to_feet_resistance_ohm = 5000)
m, i_a, i_b, i_g, v = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, v)

Again, in all respects, in arms and body, the Nth person is the one suffering the least, contrary to the popular false belief.

In [ ]:
## Example with 4 Persons, where Only the 4th is Barefoot in Wet Grass

In [ ]:
N = 3  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, hand_to_hand_resistance_ohm = 10000, neck_to_feet_resistance_ohm = 5000)
r_b[N-1] = 1
m, i_a, i_b, i_g, v = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, v)

This example is probably the most interesting since it shows that:
- yes, if the last person has much lower body to the ground resistance,
(by e.g. being the only one wearing no shoes and being in wet grass), more current can flow through his body ($i_{b,3}=74$ mA) than through some of the previous persons ($i_{b,2}=50$ mA but $i_{b,1}=100$ mA). 
- But notice that the current through the arms still lowers from the first person at the fence through the last ($i_{a,1}=224$ mA, $i_{b,2}=124$ mA, $i_{b,3}=74$ mA). This will always be the case, whatever the resistances.

# A Better Way to Play the Prank

So the belief that the last person in the chain would somehow get the most current is clearly false.
Chaining up in series would be the [Bart SImpson](https://simpsons.fandom.com/wiki/Bart_Simpson) way to play the prank. Haahaa! :)

In [ ]:
Image(filename='BartSimpsonLaughingAnimated.gif')

But if you want to play a prank on the last person in the chain, trying to get more current through any body part than through the first N-1 persons and cannot force person N to wear more conductive shoes than the N-1 previous persons are wearing, there is a way...

It's putting the N-1 persons in parallel i.o. in series of course, as in the following figure.

In [ ]:
Image(filename='ParallelChainGangOnAFence.png', width=600, height=1000)

We could write a function calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000) setting up the Kirchhoff laws, resulting in different equations here.
But it suffices to first understand that N-1 people shunted in parallel behave as a single person
of which the arm resistance is 1/(N-1) of a single person and of which the body resistance is also 1/(N-1) of a single person, as is done in the right halve of the figure above.
So the parallel N-1 people can be replaced by the equivalent network of 1 person with these 1/(N-1) lower resistances.

Now, the person N will roughly suffer N-1 times as much as the other N-1 people playing the prank.
This is clearly the way [Martin Prince Jr](https://simpsons.fandom.com/wiki/Martin_Prince) would do it! :)

In [ ]:
Image(filename='MartinPrinceJrLaughing.gif')

# Conclusion

So, let's agree that science should be taught before pranking age. :)

You can find [an article on Medium](https://logicallyyours.com/on-computation) refuting the same false belief but with as little physics / mathematical knowledge as possible. For more computational diversions from Logically Yours, have a look [at this blog](https://logicallyyours.com/on-computation). 

Peter Sels, June 20th, 2020. 
Copyright © 2020 Logically Yours BV.